In [20]:
import pandas as pd
import requests
from typing import Union
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from functools import reduce

In [21]:
def get_mf_data(url, prefix: str) -> pd.DataFrame:
    response = requests.get(url).json()['data']
    df = pd.DataFrame(response)
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

    # Set 'date' column as the index
    df.set_index('date', inplace=True)
    df = df.sort_index(ascending=True)
    # Convert 'nav' column to numeric (decimal)
    df['nav'] = pd.to_numeric(df['nav'])
    
    # df['returns'] = df['nav'].pct_change()
    df['returns'] = df['nav']/df['nav'].shift(1) - 1
    df = df.drop(df.index[0])

    # Normalize values to be relative to the first value
    # df['nav_normalized'] = (df['nav'] / df['nav'].iloc[0]) * 100

    df.columns = [prefix + col for col in df.columns]

    return df

def compute_return(df: pd.DataFrame, start_date : Union[str, pd.Timestamp], end_date : Union[str, pd.Timestamp]) -> float:
    """
    Compute the return of a mutual fund between two dates.
    """
    # Retrieve the 'nav' values for the start and end dates
    start_nav = df.loc[start_date, 'nav']
    end_nav = df.loc[end_date, 'nav']
    
    # Compute the return
    return_value = (end_nav - start_nav) / start_nav
    
    return return_value

def merge_dataframes_on_index(dfs):
    """
    Merge multiple DataFrames on their index.

    Parameters:
    dfs (list of pd.DataFrame): List of DataFrames to merge.
    how (str): Type of merge to be performed. Default is 'inner'.

    Returns:
    pd.DataFrame: Merged DataFrame.
    """
    return reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='inner'), dfs)

def plot_returns(df: pd.DataFrame, title: str):
    df['returns'].plot(figsize=(12, 8), title=title)
    plt.show()

In [22]:
icicipru_eq_and_debt_url = 'https://api.mfapi.in/mf/120251'
ppfas_fcf_url = 'https://api.mfapi.in/mf/122639'
hdfc_mco_url = 'https://api.mfapi.in/mf/118989'

icicipru_eq_and_debt = get_mf_data(icicipru_eq_and_debt_url, 'icicipru_hybrid_')
ppfas_fcf = get_mf_data(ppfas_fcf_url, 'ppfas_fcf_')
hdfc_mco = get_mf_data(hdfc_mco_url, 'hdfc_mco_')

In [23]:
result = merge_dataframes_on_index([icicipru_eq_and_debt, ppfas_fcf, hdfc_mco])

In [24]:
result

,icicipru_hybrid_nav,icicipru_hybrid_returns,ppfas_fcf_nav,ppfas_fcf_returns,hdfc_mco_nav,hdfc_mco_returns
date,,,,,,
2013-05-29,57.06,0.000526,10.0080,0.000880,18.097,0.000498
2013-05-30,57.11,0.000876,10.0327,0.002468,18.071,-0.001437
2013-05-31,56.43,-0.011907,10.0154,-0.001724,17.963,-0.005976
2013-06-03,56.33,-0.001772,10.0572,0.004174,18.057,0.005233
2013-06-04,56.29,-0.000710,10.0339,-0.002317,18.042,-0.000831
...,...,...,...,...,...,...
2024-09-27,433.21,0.000162,89.1507,-0.005000,213.612,-0.001155
2024-09-30,429.66,-0.008195,88.6165,-0.005992,213.171,-0.002064
2024-10-01,428.69,-0.002258,88.6465,0.000339,213.943,0.003622


In [25]:
ppfas_fcf

,ppfas_fcf_nav,ppfas_fcf_returns
date,,
2013-05-29,10.0080,0.000880
2013-05-30,10.0327,0.002468
2013-05-31,10.0154,-0.001724
2013-06-03,10.0572,0.004174
2013-06-04,10.0339,-0.002317
...,...,...
2024-09-27,89.1507,-0.005000
2024-09-30,88.6165,-0.005992
2024-10-01,88.6465,0.000339


KeyError: 'returns'